In [2]:
import tensorflow as tf
import os

In [3]:
file = './mini.data/train/sample-train-50-lines'
os.path.isfile(file)

True

# Two ways to iterate a SequenceExample proto

## Method 1: using `tf.data.TFRecordDataset`

In [4]:
dataset = tf.data.TFRecordDataset(file)
iterator = dataset.make_one_shot_iterator()
next_element = iterator.get_next()

In [5]:
with tf.Session() as sess:
  print(sess.run(next_element))

b'\n/\n\x17\n\x0blabel_score\x12\x08\x12\x06\n\x04\x00\x00\x80?\n\x14\n\x0blabel_index\x12\x05\x1a\x03\n\x01\x05\x12\xde\x18\n\xdb\x18\n\r0_dense_input\x12\xc9\x18\n\xc6\x18\x12\xc3\x18\n\xc0\x18\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

## Method 2: using `tf.python_io.tf_record_iterator`

In [22]:
path_to_tfrecord = file
context_features={"label_index": tf.VarLenFeature(tf.int64),
                  "label_score": tf.VarLenFeature(tf.float32)
                 }
key_dense = '0_dense_input'
# sequence_features = {key_dense: tf.FixedLenSequenceFeature((28,28), dtype=tf.float32)}
sequence_features = {key_dense: tf.VarLenFeature(dtype=tf.float32)}


for sequence_example in tf.python_io.tf_record_iterator(path_to_tfrecord):
    print('Before parsing, we only have unreadable bytes:\n', sequence_example)
    contexts, features =\
      tf.parse_single_sequence_example(sequence_example, 
                                       context_features=context_features,
                                       sequence_features=sequence_features)
    break

Before parsing, we only have unreadable bytes:
 b'\n/\n\x17\n\x0blabel_score\x12\x08\x12\x06\n\x04\x00\x00\x80?\n\x14\n\x0blabel_index\x12\x05\x1a\x03\n\x01\x05\x12\xde\x18\n\xdb\x18\n\r0_dense_input\x12\xc9\x18\n\xc6\x18\x12\xc3\x18\n\xc0\x18\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

In [7]:
features

{'0_dense_input': <tensorflow.python.framework.sparse_tensor.SparseTensor at 0x116a9d748>}

In [8]:
contexts

{'label_index': <tensorflow.python.framework.sparse_tensor.SparseTensor at 0x116a32a20>,
 'label_score': <tensorflow.python.framework.sparse_tensor.SparseTensor at 0x116a9d550>}

In [11]:
with tf.Session() as sess:
  res = sess.run(features[key_dense])
  print(res)

SparseTensorValue(indices=array([[  0,   0],
       [  0,   1],
       [  0,   2],
       ...,
       [  0, 781],
       [  0, 782],
       [  0, 783]]), values=array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.  

In [18]:
res.indices.shape

(784, 2)

In [21]:
res.indices

array([[  0,   0],
       [  0,   1],
       [  0,   2],
       ...,
       [  0, 781],
       [  0, 782],
       [  0, 783]])

In [20]:
res.values

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [23]:
res.values.shape

(784,)

In [ ]:
with tf.Session() as sess:
  print(contexts['label_index'].eval())

In [ ]:
with tf.Session() as sess:
  values = sess.run(contexts['label_index'].values)
  indices = sess.run(contexts['label_index'].indices)
  print(value, indices)

# Two ways to parse a SequenceExample proto

## Method 1: use `tf.train.SequenceExample.FromString`

In [ ]:
for sequence_example in tf.python_io.tf_record_iterator(path_to_tfrecord):
    parsed_sequence_example = tf.train.SequenceExample.FromString(sequence_example)
    print(parsed_sequence_example)
    break

In [ ]:
parsed_sequence_example.context

In [ ]:
parsed_sequence_example.feature_lists

## Method 2: use `tf.parse_single_sequence_example` 

### (see Method 2 in the previous section) 